In [1]:
from diskchef import UVFits
from pathlib import Path
import astropy.units as u
import pandas as pd

Fast (Fortran90) Mie-scattering module could not be imported. Falling back to the slower Python version.


In [2]:
IMAGER_EXEC = "bash -lc imager"

### Create fits files from observation uvtables

In [6]:
disks = ["CI Tau"]
molecules = ["CO", "13CO", "C18O"]

cubes_root = Path('observations_new')
rows_list = []
for disk in disks:
    for molecule in molecules:
        try:
            fitsfiles = (cubes_root / disk.replace(" ", "")).glob(f"{molecule}_cut.uvfits")
            path = (cubes_root / disk.replace(" ", "")).glob(f"{molecule}_cut.uvfits").__next__()
            # fitsfiles = (cubes_root / disk.replace(" ", "")).glob(f"s-Line-*-{molecule}*.uvfits")
            # path = (cubes_root / disk.replace(" ", "")).glob(f"s-Line-*-{molecule}*.uvfits").__next__()
            print(disk, molecule, path, path.exists())
            rows_list.append({"disk": disk, "molecule": molecule, "path": path})
        except StopIteration:
            path = None
            print(disk, molecule, path, False)

df = pd.DataFrame(rows_list)

CI Tau CO observations_new/CITau/CO_cut.uvfits True
CI Tau 13CO observations_new/CITau/13CO_cut.uvfits True
CI Tau C18O observations_new/CITau/C18O_cut.uvfits True


### Do not use, the fits file are cut in the channel map and we would overwrite the result

In [4]:
for _path_uvfits in df['path']:
    _molecule = df[df['path'] == _path_uvfits]['molecule'].values[0]
    print(_molecule)

    script = str("""
                        FITS "{input_file}" TO "{name}"
                        READ UV "{name}"
                        UV_MAP
                        CLEAN
                        LUT {lut}
                        ! VIEW CLEAN /NOPAUSE
                        LET SIZE 10
                        LET DO_CONTOUR NO
                        SHOW CLEAN
                        HARDCOPY "{name}.{device}" /DEVICE {device} /OVERWRITE
                        WRITE CLEAN {name}
                        FITS "{name}.fits" FROM "{name}.lmv-clean" /OVERWRITE /STYLE CASA
                """)

    UVFits.run_imaging(_path_uvfits, f"{_molecule}_cut", imager_executable=IMAGER_EXEC, script_template=script, device='png')

CO


KeyboardInterrupt: 

### Add visibility to the model fits from the corresponding observation file, and create convolved model fits files

# Probably not needed
### Create uvfits from uvt

In [ ]:
# from diskchef import UVFits
# script = str("""
#                         READ UV "{input_file}"
#                         UV_MAP
#                         CLEAN
#                         LUT {lut}
#                         ! VIEW CLEAN /NOPAUSE
#                         LET SIZE 10
#                         LET DO_CONTOUR NO
#                         SHOW CLEAN
#                         HARDCOPY "{name}.{device}" /DEVICE {device} /OVERWRITE
#                         WRITE UV "{name}"
#                         FITS "{name}.uvfits" FROM "{name}.uvt" /OVERWRITE
#
#                 """)
#
# model_visibilities = 'observations_test/DNTau/s-Line-22-CO_1+D_cut.uvt'
# output = 'observations_test/DANTau/s-Line-22-CO_1+D_cut.uvt'
# UVFits.run_imaging(model_visibilities, output, imager_executable=IMAGER_EXEC, script_template=script, device='png',)

In [ ]:
# script = str("""
#                         FITS "{input_file}" TO "{name}"
#                         READ UV "{name}"
#                         UV_MAP
#                         CLEAN
#                         LUT {lut}
#                         ! VIEW CLEAN /NOPAUSE
#                         LET SIZE 10
#                         LET DO_CONTOUR NO
#                         LET SCALE -0.05 0.45
#                         SHOW CLEAN
#                         HARDCOPY "{name}.{device}" /DEVICE {device} /OVERWRITE
#
#                 """)
#
# input = Path("/Users/franceschi/mysims/diskchef_fitting_greg/fit/best_fit_zq_4/CO_cut.fits")
# output = Path("/Users/franceschi/mysims/diskchef_fitting_greg/fit/best_fit_zq_4/CO_convolved")
# UVFits.run_imaging(input, output, imager_executable=IMAGER_EXEC, script_template=script, device='png',)